In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geone
import geone.covModel as gcm
from numba import jit

In [4]:
@jit()
def hmean(v1, v2, w1=1, w2=1):
    return (w1 + w2) / (w1 / v1 + w2 / v2)

@jit()
def mean(v1, v2, w1=1, w2=1):
    return (v1 * w1 + v2 * w2) / (w1 + w2)

In [41]:
# create functions to compute the equivalent permeability of a 2D field
# This is done by computing the arithmetic mean / or the harmonic mean in the x and y directions sequentially until the final value is obtained

def f_3D(a, dx=None, dy=None, dz=None, direction="x", typ="x"):

    """
    Function to operate a mean between two 2D fields from a 3D field

    Parameters
    ----------
    a : 3D numpy array
        The 3D field
    direction : str
        The direction of the mean to compute. Can be "x", "y" or "z"
    type : str
        The direction where fields are selected. Can be "x", "y" or "z"
    """

    # initialize weights
    w1 = 1
    w2 = 1
    new_dx = None
    new_dy = None
    new_dz = None

    if typ == "y":

        v1 = a[:, ::2, :]
        v2 = a[:, 1::2, :]
        if dy is not None:
            w1 = dy[:, ::2, :]
            w2 = dy[:, 1::2, :]
            new_dy = w1 + w2
            new_dx = dx[:, ::2, :]
            new_dz = dz[:, ::2, :]

    elif typ == "x":

        v1 = a[:, :, ::2]
        v2 = a[:, :, 1::2]
        if dx is not None:
            w1 = dx[:, :, ::2]
            w2 = dx[:, :, 1::2]
            new_dx = w1 + w2
            new_dy = dy[:, :, ::2]
            new_dz = dz[:, :, ::2]

    elif typ == "z":
            
        v1 = a[::2, :, :]
        v2 = a[1::2, :, :]
        if dz is not None:
            w1 = dz[::2, :, :]
            w2 = dz[1::2, :, :]
            new_dz = w1 + w2
            new_dx = dx[::2, :, :]
            new_dy = dy[::2, :, :]


    if typ == direction:  # series
        res = hmean(v1, v2, w1=w1, w2=w2)
    else:
        res = mean(v1, v2, w1=w1, w2=w2)
    
    return res, new_dx, new_dy, new_dz

def test_f():

    """
    Test the function f
    """
    a = np.array([[[0.51394334, 0.77316505],
                   [0.87042769, 0.00804695]],
                  [[0.30973593, 0.95760374],
                   [0.51311671, 0.31828442]]])

    a1 = f_3D(a, direction="x", typ="x")[0]
    a2 = f_3D(a1, direction="x", typ="z")[0]
    a3 = f_3D(a2, direction="x", typ="y")[0]
    res = a3[0, 0, 0]
    assert np.allclose(res, 0.3735857397539791)
    
    a1 = f_3D(a, direction="x", typ="y")[0]
    a2 = f_3D(a1, direction="x", typ="z")[0]
    a3 = f_3D(a2, direction="x", typ="x")[0]
    res = a3[0, 0, 0]
    assert np.allclose(res, 0.5323798506575812)

    print("Test passed")


# from numba import jit
# @jit(nopython=True)
def merge(v1, v2, w1, w2, normalize=True):
    
    """
    Function to merge two vectors v1 and v2 with weights w1 and w2
    Is used here to merge two columns of different sizes
    """

    if normalize:
        sum_w = w1 + w2
        w1 = w1 / sum_w
        w2 = w2 / sum_w

    return w1*v1 + w2*v2 - (w1*w2*(v1 - v2)**2)/(w1*v2 + w2*v1)

In [42]:
test_f()

Test passed


In [54]:
np.random.seed(14)
a = np.random.rand(2, 2, 2)

dx = np.ones((2, 2, 2)) 
dx[:, :, -1] = 2
a1, ndx, ndy, ndz = f_3D(a, direction="x", typ="x", dx=dx, dy=np.ones((2, 2, 2)), dz=np.ones((2, 2, 2)))
a1

array([[[0.66188498],
        [0.01201489]],

       [[0.56421699],
        [0.3644066 ]]])

In [48]:
a

array([[[0.51394334, 0.77316505],
        [0.87042769, 0.00804695]],

       [[0.30973593, 0.95760374],
        [0.51311671, 0.31828442]]])

In [55]:
np.random.seed(14)

a = np.random.rand(2, 2, 2)

a1 = f_3D(a, direction="x", typ="x")
a2 = f_3D(a1[0], direction="x", typ="z")
a3 = f_3D(a2[0], direction="x", typ="y")
print(a3[0][0, 0, 0])

a1 = f_3D(a, direction="x", typ="y")
a2 = f_3D(a1[0], direction="x", typ="z")
a3 = f_3D(a2[0], direction="x", typ="x")
print(a3[0][0, 0, 0])

0.3735857397539791
0.5323798506575812


In [57]:
test_f()

Test passed


In [334]:
def find_c_3D(k_field, direction="x", typ="min"):
    
    if typ == "min":
        if direction == "x":
            l_typ = ["x", "y", "z"]
        elif direction == "y":
            l_typ = ["y", "x", "z"]
        elif direction == "z":
            l_typ = ["z", "x", "y"]
    elif typ == "max":
        if direction == "x":
            l_typ = ["y", "z", "x"]
        elif direction == "y":
            l_typ = ["x", "z", "y"]
        elif direction == "z":
            l_typ = ["x", "y", "z"]
    
    a = np.copy(k_field)
    o = 0
    while a.flatten().shape[0] > 1:
        
        # check if the number of elements is odd
        rm_col = False
        rm_row = False
        rm_lay = False
        sim = True
        if l_typ[o] == "x":
            if a.shape[2] > 1:
                if a.shape[2] % 2 == 1:
                    # remove the last column
                    rm_col = a[:, :, -1]
                    a = a[:, :, :-1]
            else:
                sim = False

        elif l_typ[o] == "y":
            if a.shape[1] > 1:
                if a.shape[1] % 2 == 1:
                    # remove the last row
                    rm_row = a[:, -1, :]
                    a = a[:, :-1, :]
            else:
                sim = False
        elif l_typ[o] == "z":
            if a.shape[0] > 1:
                if a.shape[0] % 2 == 1:
                    # remove the last layer
                    rm_lay = a[-1, :, :]
                    a = a[:-1, :, :]
            else:
                sim = False
        if sim:
            a = f_3D(a, direction=direction, typ=l_typ[o])

        # merge the removed column, row or layer
        if rm_col is not False:
            new_col = merge(rm_col, a[:, :, -1], 1, 2)
            a = np.dstack((a, new_col.reshape(a.shape[0], a.shape[1], 1)))
        elif rm_row is not False:
            new_row = merge(rm_row, a[:, -1, :], 1, 2)
            a = np.hstack((a, new_row.reshape(a.shape[0], 1, a.shape[2])))
        elif rm_lay is not False:
            new_lay = merge(rm_lay, a[-1, :, :], 1, 2)
            a = np.vstack((a, new_lay.reshape(1, a.shape[1], a.shape[2])))

        o += 1
        if o == 3:
            o = 0

    return a[0, 0, 0]

In [241]:
find_c_3D(a, direction="x", typ="min")

[[[0.37358574]]]


0.373585739753979

In [338]:
b = np.random.rand(4, 4, 4)
find_c_3D(b, direction="x", typ="max")

0.4886625790879198

In [341]:

find_c_3D(b, direction="x", typ="min")

0.43719046483269075